<a href="https://colab.research.google.com/github/WholeHorse/SciPython/blob/main/SciPython_%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%A5%D1%83%D0%BA%D0%B0_%D0%94%D0%B6%D0%B8%D0%B2%D1%81%D0%B0_%D0%92%D0%B0%D1%80%D0%B8%D0%B0%D0%BD%D1%82_34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Работу выполнил: Бурыкин Михаил Ильич
#Группа: 3822Б1ПР3
#Задача 34. Метод Хука-Дживса:
Реализовать метод **Хука–Дживса** для минимизации многомерных функций. Экспериментально сравнить его с методами первого и второго порядка.
#Ход работы:


#Реализация метода Хука-Дживса
Для начала подключаем нужные библиотеки

In [45]:
import numpy as np
from sympy import diff, symbols, exp, evalf
from scipy import optimize
from prettytable import PrettyTable

Реализуем метод Хука-Дживса для функции: $15x-0.5y+e^{2.25x^2+2.5y^2}$ и составим таблицу.

Обозначаем функцию и другие начальные значения


In [46]:
# Значения для будущей таблицы
th = ['N',  'x_n', 'y_n', 'f(x_n, y_n)', 'sqrt(2*d**2) > d_min',]
td = []

# Начальная точка, шаг и критерий остановки
x_n = 1
d_min = 0.1
d = 1
y_n = 2

# Функция
def f(x,y):
    f = 15*x-0.5*y+np.exp(2.25*(x**2)+2.5*(y**2))
    return f

f_n = f(x_n,y_n)


Реализуем исследующий поиск (+-d приращение координат, если функция уменьшилась, то запоминаем координаты)

In [47]:
def search(x, y, f0, d):
    if f(x + d, y) < f0:
        f0 = f(x + d, y)
        x += d
    elif f(x - d, y) <f0: 
        f0 = f(x - d, y)
        x -= d
    if f(x, y + d) < f0:
        f0 = f(x, y + d)
        y += d
    elif f(x, y - d) < f0: 
        f0 = f(x, y - d)
        y -= d
    return x, y, f0

Образец поиска: идет шаг от начальной точки по прямой, идущей с предыдущей начальной точки

Если функция уменьшилась, запоминаем точку и продолжаем
, если нет, возвращаемся к посл. удачной точки

Если в результате исследования функция не изменилась шаг уменьшаем в 2 раза

Выполняем, пока корень из суммы квадратов шага > критерия остановки

In [48]:
n_iter = 1
while np.sqrt(2*d**2) > d_min:
    # Вывод данных в таблицу
    td.append(n_iter)
    td.append(x_n)
    td.append(y_n)
    td.append(round(f_n, 3))
    
    x, y, f0 = search(x_n, y_n, f_n, d)
    if f0 == f_n:
        d /= 2
    while f0 < f_n:
        # Образец поиска
        f_n = f0
        x_t = x + (x - x_n)
        y_t = y + (y - y_n)        
        if f(x_t, y_t) < f_n:
            x_n, y_n = x_t, y_t                                 
        else:
            x_n, y_n = x, y
        f0 = f(x_n,y_n)
    f_n = f0
    n_iter += 1

    sq = round(np.sqrt(2*d**2), 3)
    td.append(str(sq) + ' ? ' + str(d_min) + ' = ' + str(sq > d_min))

Подсчитываем кол-во столбцов на будущее и определяем таблицу

In [49]:
columns = len(th)
table = PrettyTable(th)

Копируем список td, на случай если он будет использоваться в коде дальше

Далее входим в цикл который заполняет нашу таблицу

In [50]:
td_data = td[:]

while td_data:
    table.add_row(td_data[:columns])
    td_data = td_data[columns:]

print(table)
print(x_n,y_n)
print(round(f(x_n,y_n), 3))

+---+-------+-----+-------------+----------------------+
| N |  x_n  | y_n | f(x_n, y_n) | sqrt(2*d**2) > d_min |
+---+-------+-----+-------------+----------------------+
| 1 |   1   |  2  |  208995.289 |  1.414 ? 0.1 = True  |
| 2 |   0   |  1  |    11.682   |  1.414 ? 0.1 = True  |
| 3 |   0   |  0  |     1.0     |  1.414 ? 0.1 = True  |
| 4 |   -1  |  0  |    -5.512   |  0.707 ? 0.1 = True  |
| 5 |   -1  |  0  |    -5.512   |  0.707 ? 0.1 = True  |
| 6 |  -0.5 |  0  |    -5.745   |  0.354 ? 0.1 = True  |
| 7 |  -0.5 |  0  |    -5.745   |  0.354 ? 0.1 = True  |
| 8 | -0.75 |  0  |    -7.705   |  0.177 ? 0.1 = True  |
| 9 | -0.75 |  0  |    -7.705   | 0.088 ? 0.1 = False  |
+---+-------+-----+-------------+----------------------+
-0.75 0
-7.705


# Реаилзация метода Ньютона

Теперь для сравнения с другим методом мне нужно реализовать метод Ньютона, который является методом второго порядка, для функции: $15x-0.5y+e^{2.25x^2+2.5y^2}$ и составить таблицу


In [51]:
# Значения для будущей таблицы
th = ['N',  'x_n', 'y_n', 'f(x_n, y_n)',]
td = []

# Начальная точка, шаг и критерий остановки
x, y = symbols('x y')
d_min = 0.1
d = 1
x_n = 1
y_n = 2

# Функция
def fun(x,y):
    f = 15*x-0.5*y+exp(2.25*(x**2)+2.5*(y**2))
    return f

Далее нам следует найти Градиенты по x и y и сделать проверку (Сумма квадратов значений градиентов < Критерия)

In [52]:
def grad_x(x, y):
    f = diff(15*x-0.5*y+exp(2.25*(x**2)+2.5*(y**2)),x)
    return f

def grad_y(x, y):
    f = diff(15*x-0.5*y+exp(2.25*(x**2)+2.5*(y**2)),y)
    return f

def check(x,y):
    return np.sqrt(x**2+y**2)

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

Градиенты в нач. точке, матрица Гессе, обратный гессиан. Новая точка минимума, как прошла точка обратная гессиан * матрицу градиентов

Если матрица Гессе определена положительно без критерия остановки,
то фиксируем новую точку и делаем повтор действий 

In [53]:
n_iter = 1
while True:
    td.append(n_iter)
    td.append(round(x_n, 3))
    td.append(round(y_n, 3))
    td.append(round(fun(x_n,y_n), 3))
    
    f_n_x = (grad_x(x, y))
    f_n_y = (grad_y(x, y))
    f_nx = float(f_n_x.evalf(subs={x:x_n,y:y_n}))
    f_ny = float(f_n_y.evalf(subs={x:x_n,y:y_n}))
    
    m = np.array([[f_nx],[f_ny]])
    
    x1 = float(diff(grad_x(x,y),x).evalf(subs={x:x_n,y:y_n}))
    x2 = float(diff(grad_x(x,y),y).evalf(subs={x:x_n,y:y_n}))
    x3 = float(diff(grad_y(x,y),x).evalf(subs={x:x_n,y:y_n}))
    x4 = float(diff(grad_y(x,y),y).evalf(subs={x:x_n,y:y_n}))
    ges = np.array([[x1,x2],[x3,x4]])
    ges_inv = np.linalg.inv(ges)
    x0 = np.array([[x_n],[y_n]])-(ges_inv).dot(m)
    
    m1 = float(f_n_x.evalf(subs={x:x0[0][0],y:x0[1][0]}))
    m2 = float(f_n_y.evalf(subs={x:x0[0][0],y:x0[1][0]}))
    x_n = x0[0][0]
    y_n = x0[1][0]
    if is_pos_def(ges) and check(m1,m2)>d_min:
        pass     
    else:
        break
    n_iter += 1
    

td.append(n_iter+1)
td.append(round(x_n, 3))
td.append(round(y_n, 3))
td.append(round(fun(x_n,y_n), 3))

Подсчитываем кол-во столбцов на будущее и определяем таблицу

In [54]:
columns = len(th)
table = PrettyTable(th)

Копируем список td, на случай если он будет использоваться в коде дальше

Далее входим в цикл который заполняет нашу таблицу

In [55]:
td_data = td[:]
while td_data:
    table.add_row(td_data[:columns])
    td_data = td_data[columns:]

print(table)
print(round(x_n, 3), round(y_n, 3))
print(round(fun(x_n,y_n), 3))

+----+--------+-------+-------------+
| N  |  x_n   |  y_n  | f(x_n, y_n) |
+----+--------+-------+-------------+
| 1  |   1    |   2   |  208995.289 |
| 2  | 0.961  | 1.922 |  81488.497  |
| 3  |  0.92  |  1.84 |  31922.566  |
| 4  | 0.878  | 1.756 |  12576.181  |
| 5  | 0.833  | 1.667 |   4989.418  |
| 6  | 0.786  | 1.575 |   1997.826  |
| 7  | 0.736  | 1.478 |   810.397   |
| 8  | 0.682  | 1.377 |   335.209   |
| 9  | 0.619  | 1.271 |   142.950   |
| 10 |  0.54  | 1.162 |    63.813   |
| 11 |  0.43  | 1.052 |    30.050   |
| 12 | 0.247  | 0.949 |    14.124   |
| 13 | -0.089 | 0.842 |    4.246    |
| 14 | -0.619 | 0.622 |    -3.374   |
| 15 | -0.817 | 0.298 |    -6.796   |
| 16 | -0.821 | 0.101 |    -7.691   |
| 17 | -0.801 | 0.032 |    -7.784   |
| 18 | -0.797 | 0.024 |    -7.786   |
+----+--------+-------+-------------+
-0.797 0.024
-7.786


# Реализация простейшего градиентного метода

Так же для сравнения следует реализовать простейший градиентный метод, который является методом первого порядка, для функции: $15x-0.5y+e^{2.25x^2+2.5y^2}$ и составить таблицу

In [56]:
# Таблица
th = ['N',  'x_n', 'y_n', 'f(x_n, y_n)', 'Норма градиента x', 'Норма градиента y',]
td = []

# Начальная точка, проверка для остановки и мелкий шаг
x, y = symbols('x y')
d_min = 0.0001
d = 0.01
x_n = 1
y_n = 2

# Функция
def fun(par):
    x, y = par
    return 15*x-0.5*y+exp(2.25*(x**2)+2.5*(y**2))

Находим производные по x и y и делаем проверку (Сумма квадратов значений градиентов < критерия)

In [57]:
def grad_x(x, y):
    return diff(15*x-0.5*y+exp(2.25*(x**2)+2.5*(y**2)),x)

def grad_y(x, y):
    return diff(15*x-0.5*y+exp(2.25*(x**2)+2.5*(y**2)),y)

def check(x,y):
    return np.sqrt(x**2+y**2)

Вычисляем градиенты в базовой точке, проверяем критерии остановки, для быстрой сходимости, т.к. не указано количество итераций, добавляем критерий на минимальное допустимое значение разницы между старыми и новыми координатами

Новая базовая точка - вычитаем из старой шаг * градиент  

In [58]:
n_iter = 1
while True:
    td.append(n_iter)
    td.append(x_n)
    td.append(y_n)
    td.append(fun([x_n,y_n]))
    
    f_n_x = (grad_x(x, y))
    f_n_y = (grad_y(x, y))
    f_nx = float(f_n_x.evalf(subs={x:x_n,y:y_n}))
    f_ny = float(f_n_y.evalf(subs={x:x_n,y:y_n}))

    td.append(abs(f_nx))
    td.append(abs(f_ny))
    
    if check(f_nx,f_ny) < d_min or (d * f_nx < d_min and d * f_ny < d_min):
        break    
    x_n -= d * f_nx/abs(f_nx)
    y_n -= d * f_ny/abs(f_ny)
    n_iter += 1


td.append(n_iter+1)
td.append(x_n)
td.append(y_n)
td.append(fun([x_n,y_n]))
td.append(abs(f_nx))
td.append(abs(f_ny))

Подсчитываем кол-во столбцов на будущее и определяем таблицу

Копируем список td, на случай если он будет использоваться в коде дальше

Далее входим в цикл который заполняет нашу таблицу

In [59]:
columns = len(th)
table = PrettyTable(th)

td_data = td[:]

while td_data:
    table.add_row(td_data[:columns])
    td_data = td_data[columns:]

print(table)
print(x_n,y_n)
print(fun([x_n,y_n]))

+-----+------------------------+----------------------+--------------------+----------------------+---------------------+
|  N  |          x_n           |         y_n          |    f(x_n, y_n)     |  Норма градиента x   |  Норма градиента y  |
+-----+------------------------+----------------------+--------------------+----------------------+---------------------+
|  1  |           1            |          2           |  208995.288869713  |  940430.7999137084   |  2089812.3886971297 |
|  2  |          0.99          |         1.99         |  180873.216512069  |  805743.4555362671   |  1799550.1470450861 |
|  3  |          0.98          |         1.98         |  156684.186060139  |  690931.7994252107   |  1551037.2129953708 |
|  4  |          0.97          |         1.97         |  135859.270189907  |   592981.503153945   |  1338079.696120586  |
|  5  |          0.96          |         1.96         |  117914.353002339  |  509347.03057010693  |  1155428.6434229277 |
|  6  |          0.95   

# Вывод
В результате, мы получаем примерно одинаковые значения, но количество итераций в данном случае различается, из чего можно сделать вывод, что метод Хука-Дживса работает более быстро, ведь он относится к методам нулевого порядка и не требует вычисления производных, в отличии от методов Ньютона и Простейшего градиентного



|      Название метода   | x    | y  | f(x,y) | Количество итераций|
|------          |------|----|-------||
| Метод Хука - Дживса: |-0.75|   0 | -7.705 | 9|
| Метод Ньютона: |   -0.797   | 0.024   | -7.786| 18|
| Метод Простейший градиентный: |   -0.8000000000000013   | 0.01999999999999836   |   -7.78508137613492     |200|

